<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#ResMiCo-sm" data-toc-modified-id="ResMiCo-sm-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>ResMiCo-sm</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Config" data-toc-modified-id="Config-4.0.1"><span class="toc-item-num">4.0.1&nbsp;&nbsp;</span>Config</a></span></li><li><span><a href="#Run" data-toc-modified-id="Run-4.0.2"><span class="toc-item-num">4.0.2&nbsp;&nbsp;</span>Run</a></span></li></ul></li></ul></li><li><span><a href="#metaquast" data-toc-modified-id="metaquast-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>metaquast</a></span></li><li><span><a href="#minimap2" data-toc-modified-id="minimap2-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>minimap2</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Convert-to-tsv" data-toc-modified-id="Convert-to-tsv-6.0.1"><span class="toc-item-num">6.0.1&nbsp;&nbsp;</span>Convert to tsv</a></span></li><li><span><a href="#Loading-hits" data-toc-modified-id="Loading-hits-6.0.2"><span class="toc-item-num">6.0.2&nbsp;&nbsp;</span>Loading hits</a></span></li></ul></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* ResMiCo-sm on CAMI contigs

# Var

In [ ]:
work_dir = '/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM_cami-contigs/'



# Init

In [4]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(tidytable)
library(LeyLabRMisc)
Init()

In [28]:
clustermq_setup()

# ResMiCo-sm

### Config

In [6]:
F = file.path(work_dir, 'config.yaml')
cat_file(F)

# Input table
## Table of genomes
genomes_file: /ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM_cami-contigs/samples.tsv

# Output directory
output_dir: /ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM_cami-contigs/

# Temporary output directory (/dev/shm/ for shared memory)
#tmp_dir: /tmp/
tmp_dir: /ebio/abt3_scratch/

# Email for notifications ("Skip" to skip)
email: nyoungblut@tuebingen.mpg.de

# software parameters for data simulations
## Use "Skip" to skip  steps. If no params for rule, use ""
## `reps:` is the number of replicate simulations
## If `keep_reads: True`, then simulated reads are kept
params:
  ionice: -c 3
  # simulating metagenomes
  MGSIM:
#    genome_download: ""
    sim_reps: 1
    community:
      richness:
        - 0.5
        - 0.8
      abundance_distribution:
        - "mean:10,sigma:1"
        - "mean:10,sigma:2"
      random_seed: 8192
      other_sim_params: --group-bias 0
    reads: 
     

### Run

```
(snakemake) @ rick:/ebio/abt3_projects/DeepMAsED/version2/deepmased/ResMiCo-SM
screen -L -S ./snakemake_sge.sh /ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM_cami-contigs/config.yaml 30 -F
```

# metaquast

* running metaQUAST 'manually' due to non-standard workflow: "real" contigs + references
* method
  * combine ref genomes
  * metaquast
    * cami contigs, ref genomes, reads

In [39]:
# var
base_dir = '/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/'

ref_genomes_table = file.path(base_dir, 'genomes.tsv')

mq_dir = file.path(base_dir, 'metaquast')
contigs_file = file.path(mq_dir, 'samples.tsv')

ref_genomes_dir = file.path(base_dir, 'genomes')

exe_file = '/ebio/abt3_projects/DeepMAsED/version2/deepmased/ResMiCo-SM/bin/scripts/metaquast_short.py'

# created conda env for work
## mamba create -p conda_env/metaquast bioconda::quast=5.0.2 r-renv r-glue r-clustermq
conda_env = file.path(mq_dir, 'conda_env', 'metaquast')

In [40]:
# Table of CAMI contigs + associated data
contigs = Fread(contigs_file)
contigs

Taxon,Fasta,Sample,Read1,Read2
<chr>,<chr>,<chr>,<chr>,<chr>
2017.12.04_18.45.54_sample_0,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/2017.12.04_18.45.54_sample_0/contigs/anonymous_gsa.fasta.gz,2017.12.04_18.45.54_sample_0,/ebio/abt3_scratch/nyoungblut/CAMI/CAMI2_HMP-gut/2017.12.04_18.45.54_sample_0_R1.fq,/ebio/abt3_scratch/nyoungblut/CAMI/CAMI2_HMP-gut/2017.12.04_18.45.54_sample_0_R2.fq
2017.12.04_18.45.54_sample_1,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/2017.12.04_18.45.54_sample_1/contigs/anonymous_gsa.fasta.gz,2017.12.04_18.45.54_sample_1,/ebio/abt3_scratch/nyoungblut/CAMI/CAMI2_HMP-gut/2017.12.04_18.45.54_sample_1_R1.fq,/ebio/abt3_scratch/nyoungblut/CAMI/CAMI2_HMP-gut/2017.12.04_18.45.54_sample_1_R2.fq
⋮,⋮,⋮,⋮,⋮
2017.12.04_18.45.54_sample_5,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/2017.12.04_18.45.54_sample_5/contigs/anonymous_gsa.fasta.gz,2017.12.04_18.45.54_sample_5,/ebio/abt3_scratch/nyoungblut/CAMI/CAMI2_HMP-gut/2017.12.04_18.45.54_sample_5_R1.fq,/ebio/abt3_scratch/nyoungblut/CAMI/CAMI2_HMP-gut/2017.12.04_18.45.54_sample_5_R2.fq
2017.12.04_18.45.54_sample_9,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/2017.12.04_18.45.54_sample_9/contigs/anonymous_gsa.fasta.gz,2017.12.04_18.45.54_sample_9,/ebio/abt3_scratch/nyoungblut/CAMI/CAMI2_HMP-gut/2017.12.04_18.45.54_sample_9_R1.fq,/ebio/abt3_scratch/nyoungblut/CAMI/CAMI2_HMP-gut/2017.12.04_18.45.54_sample_9_R2.fq


In [61]:
bash_job = function (cmd, conda_env = NULL, stdout = TRUE, stderr = TRUE, 
                 print_output = TRUE, return_output = FALSE, log_file = NULL, 
                 verbose = TRUE, wait = TRUE){
    for (x in c(stderr, stdout)) {
        if (is.character(stderr)) {
            dir.create(dirname(x), recursive = TRUE, showWarnings = FALSE)
        }
    }
    if (!is.null(conda_env)) {
        CMD = sprintf(". ~/.bashrc && conda activate %s &&", 
            conda_env)
    }
    else {
        CMD = ""
    }
    CMD = sprintf("%s %s", CMD, cmd)
    CMD = sprintf("-c \"%s\"", CMD)
    if (verbose == TRUE) {
        message(sprintf("bash %s", CMD))
    }
    ret = system2("bash", CMD, stdout = stdout, stderr = stderr, 
        wait = TRUE)
    if (print_output == TRUE) {
        cat(paste(ret, collapse = "\n"))
    }
    if (return_output == TRUE) {
        return(ret)
    }
    return(invisible())
}

In [41]:
#' run metaquast on samples
run_mq = function(x, exe_file, ref_dir, out_dir, conda_env){
    cmd = '{exe} -t 8 --max-ref-number 0 --min-contig 1000 \
    --min-identity 95 --extensive-mis-size 100 \
    --no-html --no-plots --no-icarus --no-gc --no-snps --no-sv \
    --max-ref-number 0 --no-gzip -o {out_dir} -r {ref_dir} {contigs}'
    cmd = gsub('\n', ' ', cmd)
    out_dir = file.path(out_dir, x$Taxon)
    cmd = glue::glue(cmd, exe=exe_file, out_dir=out_dir, ref_dir=ref_dir, contigs=x$Fasta)
    bash_job(cmd, conda_env = conda_env)
}

tmpl = list(job_time = '24:00:00', job_mem = '4G', cores = 8, log_file = clustermq_logfile())  
contigs %>%
    group_split.(Taxon) %>%
    Q(run_mq, x=., exe_file=exe_file, ref_dir=ref_genomes_dir, out_dir=mq_dir, 
      conda_env=quast_conda_env, n_jobs=10, job_size=1, template=tmpl, 
      pkgs=c('glue'), export=list(bash_job=bash_job))

Setting logfile dir: /ebio/abt3_scratch//nyoungblut/clustermq/7d48b5d8-e6de-4379-baf0-a181fceb4746

Submitting 10 worker jobs (ID: cmq6386) ...

Running 10 calculations (0 objs/0 Mb common; 1 calls/chunk) ...

Warning message in qsys$receive_data(timeout = timeout):
“
Version mismatch: master has 0.8.95.2, worker 0.8.95.1”

[---------------------------------------------------]   0% (1/10 wrk) eta:  ?s

[---------------------------------------------------]   0% (2/10 wrk) eta:  ?s

[---------------------------------------------------]   0% (3/10 wrk) eta:  ?s

[---------------------------------------------------]   0% (4/10 wrk) eta:  ?s

[---------------------------------------------------]   0% (5/10 wrk) eta:  ?s

[---------------------------------------------------]   0% (6/10 wrk) eta:  ?s

[---------------------------------------------------]   0% (7/10 wrk) eta:  ?s

[---------------------------------------------------]   0% (8/10 wrk) eta:  ?s

[---------------------------------

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

In [42]:
send_email('http://127.0.0.1:9704/notebooks/version2/deepmased/notebooks/06_version2/01_simulation_datasets/04_CAMI/01a_CAMI2_HMP-gut_contigs.ipynb')

Sending email to: nyoungblut@tuebingen.mpg.de



# minimap2

* Find similar contigs for the CAMI2-provided contigs versus the de-novo contigs generated with the reads

In [62]:
denovo_contigs_file = '/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM/assembly/contigs.tsv'


In [63]:
denovo_contigs = Fread(denovo_contig_file)
denovo_contigs

Sample,Fasta
<chr>,<chr>
2017.12.04_18.45.54_sample_2,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM/assembly/2017.12.04_18.45.54_sample_2/NA/NA/NA/NA/megahit/contigs_filtered.fasta.gz
2017.12.04_18.45.54_sample_2,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM/assembly/2017.12.04_18.45.54_sample_2/NA/NA/NA/NA/metaspades/contigs_filtered.fasta.gz
⋮,⋮
2017.12.04_18.45.54_sample_10,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM/assembly/2017.12.04_18.45.54_sample_10/NA/NA/NA/NA/metaspades/contigs_filtered.fasta.gz
2017.12.04_18.45.54_sample_10,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM/assembly/2017.12.04_18.45.54_sample_10/NA/NA/NA/NA/megahit/contigs_filtered.fasta.gz


In [71]:
# checking overlap
overlap(denovo_contigs$Sample, contigs$Sample)
contigs_j = contigs %>%
    rename.('provided_contigs' = Fasta) %>%
    inner_join.(denovo_contigs %>% 
                    rename.('denovo_contigs' = Fasta), 
                c('Sample')) %>%
    select.(Sample, provided_contigs, denovo_contigs) %>%
    mutate.(Assembler = gsub('.+/NA/(megahit|metaspades)/.+', '\\1', denovo_contigs))
contigs_j

intersect(x,y): 10 
setdiff(x,y): 0 
setdiff(y,x): 0 
union(x,y): 10 


Sample,provided_contigs,denovo_contigs,Assembler
<chr>,<chr>,<chr>,<chr>
2017.12.04_18.45.54_sample_2,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/2017.12.04_18.45.54_sample_2/contigs/anonymous_gsa.fasta.gz,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM/assembly/2017.12.04_18.45.54_sample_2/NA/NA/NA/NA/megahit/contigs_filtered.fasta.gz,megahit
2017.12.04_18.45.54_sample_2,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/2017.12.04_18.45.54_sample_2/contigs/anonymous_gsa.fasta.gz,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM/assembly/2017.12.04_18.45.54_sample_2/NA/NA/NA/NA/metaspades/contigs_filtered.fasta.gz,metaspades
⋮,⋮,⋮,⋮
2017.12.04_18.45.54_sample_10,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/2017.12.04_18.45.54_sample_10/contigs/anonymous_gsa.fasta.gz,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM/assembly/2017.12.04_18.45.54_sample_10/NA/NA/NA/NA/metaspades/contigs_filtered.fasta.gz,metaspades
2017.12.04_18.45.54_sample_10,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/2017.12.04_18.45.54_sample_10/contigs/anonymous_gsa.fasta.gz,/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM/assembly/2017.12.04_18.45.54_sample_10/NA/NA/NA/NA/megahit/contigs_filtered.fasta.gz,megahit


In [76]:
#' run metaquast on samples
run_minimap = function(x, out_dir, conda_env){
    D = file.path(out_dir, x$Sample, x$Assembler)
    if(! dir.exists(D)){
        dir.create(D, recursive = TRUE)
    }
    out_paf = file.path(D, 'hits.paf')
    cmd = 'minimap2 -t 8 -k 18 -x asm5 {ref} {query} > {out_paf}'
    cmd = glue::glue(cmd, ref=x$provided_contigs, query=x$denovo_contigs, out_sam=out_sam)
    bash_job(cmd, conda_env = conda_env)
}
out_dir = file.path(work_dir, 'minimap2')
tmpl = list(job_time = '00:59:00', job_mem = '5G', cores = 8, log_file = clustermq_logfile())  
contigs_j %>%
    group_split.(Sample, Assembler) %>%
    Q(run_minimap, x=., out_dir=out_dir,
      conda_env=quast_conda_env, n_jobs=20, job_size=1, template=tmpl, 
      pkgs=c('glue'), export = list('bash_job' = bash_job))

Setting logfile dir: /ebio/abt3_scratch//nyoungblut/clustermq/cef05831-cc1b-4b78-a84b-7d3e740225a3

Submitting 20 worker jobs (ID: cmq9424) ...

Running 20 calculations (1 objs/0 Mb common; 1 calls/chunk) ...

Warning message in qsys$receive_data(timeout = timeout):
“
Version mismatch: master has 0.8.95.2, worker 0.8.95.1”

[---------------------------------------------------]   0% (1/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (2/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (3/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (4/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (5/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (6/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (7/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (8/20 wrk) eta:  ?s

[---------------------------------

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

### Convert to tsv

In [95]:
conda_env = '/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/metaquast/conda_env/metaquast'

In [96]:
#' run metaquast on samples
run_paf2tsv = function(x, out_dir, exe, conda_env){
    D = file.path(out_dir, x$Sample, x$Assembler)
    in_paf = file.path(D, 'hits.paf')
    out_tsv = file.path(D, 'hits.tsv')
    cmd = '{exe} {in_paf} > {out_tsv}'
    cmd = glue::glue(cmd, exe=exe, in_paf=in_paf, out_tsv=out_tsv)
    bash_job(cmd, conda_env = conda_env)
}
out_dir = file.path(work_dir, 'minimap2')
exe = file.path(out_dir, 'paf2tsv.py')
tmpl = list(job_time = '00:59:00', job_mem = '7G', cores = 1, log_file = clustermq_logfile())  
contigs_j %>%
    group_split.(Sample, Assembler) %>%
    Q(run_paf2tsv, x=., out_dir=out_dir, exe=exe,
      conda_env=conda_env, n_jobs=20, job_size=1, template=tmpl, 
      pkgs=c('glue'), export = list('bash_job' = bash_job))

Setting logfile dir: /ebio/abt3_scratch//nyoungblut/clustermq/b618a5df-0719-4c06-8997-d3fd33d89175

Submitting 20 worker jobs (ID: cmq7363) ...

Running 20 calculations (1 objs/0 Mb common; 1 calls/chunk) ...

Warning message in qsys$receive_data(timeout = timeout):
“
Version mismatch: master has 0.8.95.2, worker 0.8.95.1”

[---------------------------------------------------]   0% (1/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (2/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (3/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (4/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (5/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (6/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (7/20 wrk) eta:  ?s

[---------------------------------------------------]   0% (8/20 wrk) eta:  ?s

[---------------------------------

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

### Loading hits

In [119]:
base_dir = file.path(work_dir, 'minimap2')
hit_files = list_files(base_dir, 'hits.tsv')
hit_files %>% length
hit_files %>% head(n=3)

[1] 20

[1] "/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM_cami-contigs//minimap2/2017.12.04_18.45.54_sample_0/megahit/hits.tsv"   
[2] "/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM_cami-contigs//minimap2/2017.12.04_18.45.54_sample_0/metaspades/hits.tsv"
[3] "/ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM_cami-contigs//minimap2/2017.12.04_18.45.54_sample_1/megahit/hits.tsv"

In [120]:
# loading hits
read_hits = function(F){
    sample = gsub('.+/minimap2/', '', F)
    sample = gsub('/.+', '', sample)
    dt = Fread(F)
    dt$sample = sample
    return(dt)
}

hits = hit_files %>%
    lapply(read_hits) %>%
    rbindlist(use.names=TRUE)
hits

query_name,target_name,query_coverage,target_coverage,seq_id,sample
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
k141_17,S0C12831,0.9404762,0.0005045236,0.9088608,2017.12.04_18.45.54_sample_0
k141_19,S0C51621,0.9250624,0.0033958296,0.8145815,2017.12.04_18.45.54_sample_0
⋮,⋮,⋮,⋮,⋮,⋮
NODE_23666_length_1000_cov_1.320635,S9C13303,0.949,0.2398888,0.8398314,2017.12.04_18.45.54_sample_9
NODE_23667_length_1000_cov_1.278307,S9C6844,0.975,0.4831516,0.8379487,2017.12.04_18.45.54_sample_9


In [121]:
hits$query_name %>% unique_n('queries')
hits$target_name %>% unique_n('targets')

No. of unique queries: 286107 
No. of unique targets: 35566 


In [122]:
# duplicates
dup_contigs = hits %>%
    filter.(seq_id >= 0.99,  query_coverage >= 0.95, target_coverage >= 0.95)
dup_contigs

query_name,target_name,query_coverage,target_coverage,seq_id,sample
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
k141_31037,S0C67912,0.9997810,0.9997810,0.9938668,2017.12.04_18.45.54_sample_0
NODE_124_length_68478_cov_9.339374,S0C67912,0.9997663,0.9995182,0.9948146,2017.12.04_18.45.54_sample_0
⋮,⋮,⋮,⋮,⋮,⋮
NODE_789_length_9948_cov_18.842313,S9C1442,0.9976880,0.9976880,0.9943577,2017.12.04_18.45.54_sample_9
NODE_996_length_8057_cov_17.292677,S9C6488,0.9991312,0.9991312,0.9954037,2017.12.04_18.45.54_sample_9


In [125]:
# listing all contigs
df.dims(30)
dup_contigs
df.dims()

query_name,target_name,query_coverage,target_coverage,seq_id,sample
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
k141_31037,S0C67912,0.9997810,0.9997810,0.9938668,2017.12.04_18.45.54_sample_0
NODE_124_length_68478_cov_9.339374,S0C67912,0.9997663,0.9995182,0.9948146,2017.12.04_18.45.54_sample_0
NODE_227_length_35816_cov_106.334051,S0C18644,0.9979339,0.9551577,0.9926977,2017.12.04_18.45.54_sample_0
k141_22355,S1C21631,0.9997081,0.9997081,0.9952407,2017.12.04_18.45.54_sample_1
k141_82026,S1C57230,0.9996064,0.9994551,0.9950324,2017.12.04_18.45.54_sample_1
NODE_93_length_68517_cov_23.007683,S1C21631,0.9997081,0.9997081,0.9952407,2017.12.04_18.45.54_sample_1
k141_19273,S10C4782,0.9990072,0.9990072,0.9954037,2017.12.04_18.45.54_sample_10
NODE_1091_length_8058_cov_12.889666,S10C4782,0.9990072,0.9990072,0.9954037,2017.12.04_18.45.54_sample_10
k141_19646,S11C2462,0.9999632,0.9997974,0.9943256,2017.12.04_18.45.54_sample_11


In [126]:
# writing table
F = file.path(work_dir, 'dup_contigs.tsv')
write_table(dup_contigs, F)

File written: /ebio/abt3_projects/databases_no-backup/CAMI/CAMI2_HMP-gut/short_read/ResMiCo-SM_cami-contigs//dup_contigs.tsv 


In [118]:
# duplicates (less stringent)
dup_contigs = hits %>%
    filter.(seq_id >= 0.98,  query_coverage >= 0.5, target_coverage >= 0.5)
dup_contigs %>%
    summarize.(n = n.(), .by=c(sample)) %>%
    .$n %>% summary_x('contigs per sample')
dup_contigs

,Min.,1st Qu.,Median,Mean,3rd Qu.,Max.,sd,sd_err_of_mean
contigs per sample,6,11.25,12.5,19.7,29,46,13.35,5.45


query_name,target_name,query_coverage,target_coverage,seq_id,sample
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
k141_15804,S0C28727,0.9743447,0.5233281,0.9834001,2017.12.04_18.45.54_sample_0
k141_18211,S0C53064,0.7812736,0.6138734,0.9892289,2017.12.04_18.45.54_sample_0
⋮,⋮,⋮,⋮,⋮,⋮
NODE_789_length_9948_cov_18.842313,S9C1442,0.9976880,0.9976880,0.9943577,2017.12.04_18.45.54_sample_9
NODE_996_length_8057_cov_17.292677,S9C6488,0.9991312,0.9991312,0.9954037,2017.12.04_18.45.54_sample_9


# sessionInfo

In [89]:
sessionInfo()

R version 4.1.1 (2021-08-10)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 18.04.6 LTS

Matrix products: default
BLAS/LAPACK: /tmp/global2/nyoungblut/code/general/conda_envs/general-tidyverse/lib/libopenblasp-r0.3.18.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] clustermq_0.8.95.2 LeyLabRMisc_0.2.1  tidytable_0.6.6    data.table_1.14.2 
[5] ggplot2_3.3.5      tidyr_1.1.4        dplyr_1.0.7       

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.7        plyr_1.8.6        pillar_1.6.4      compiler_4.1.1   
 [5] prettyu